In [13]:
import cdsapi
import numpy as np
import xarray as xr

c = cdsapi.Client()

Via the [Copernicus Climate Change Service](https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-agroclimatic-indicators?tab=overview):

"This dataset provides agroclimatic indicators used to characterise plant-climate interactions for global agriculture. Agroclimatic indicators are useful in conveying climate variability and change in the terms that are meaningful to the agricultural sector. The objective of this dataset is to provide these indicators at a global scale in an easily accessible and usable format for further downstream analysis and the forcing of agricultural impact models.

ERA-interim reanalysis and bias-corrected climate datasets have been used to generate the agroclimatic indicators for historical and future time periods. The input data was provided through the Inter-Sectoral Impact Model Intercomparison Project (ISIMIP), of which the ISIMIP Fast Track product was used. This product contains daily, biased-corrected, climate data from 5 CMIP5 General Circulation Models covering the period 1951-2099 (historical run up to 2005). The agroclimatic indicators were also generated using the WFDEI (Watch Forcing Data methodology applied to ERA-Interim) for the 1981-2010 climatological period. A total of 26 indicators are provided in this dataset at a spatial resolution of 0.5°x0.5° on a lat-lon grid. The temporal resolution of the variables differs depending on the indicator - they are available at 10 consecutive days (10-day), seasonal or annual resolution.

Agroclimatic indicators are often used in species distribution modelling to study phenological developments of plants under varying climate conditions. For many users in the agricultural community, assessments of crop development for the current or future cropping seasons are particularly important. This is especially true for the agro-policy and the agro-business communities, as early indications of production anomalies are of paramount importance for tax/subsidies and price volatility. The provision of pre-computed agroclimatic indicators make them readily available to the user and will facilitate the use of climate data by the agricultural community."

First - collect the fitted values for historical data:

In [135]:
c.retrieve(
    'sis-agroclimatic-indicators',
    {
        'format': 'tgz',
        'variable': [
            'biologically_effective_degree_days', 'frost_days', 'heavy_precipitation_days',
            'ice_days', 'maximum_of_daily_maximum_temperature', 'maximum_of_daily_minimum_temperature',
            'mean_of_daily_maximum_temperature', 'mean_of_daily_mean_temperature', 'mean_of_daily_minimum_temperature',
            'mean_of_diurnal_temperature_range', 'minimum_of_daily_maximum_temperature', 'minimum_of_daily_minimum_temperature',
            'precipitation_sum', 'simple_daily_intensity_index', 'summer_days',
            'tropical_nights', 'very_heavy_precipitation_days', 'wet_days',
        ],
        'origin': 'gfdl_esm2m_model',
        'experiment': 'historical',
        'temporal_aggregation': '10_day',
        'period': [
            '195101_198012', '198101_201012',
        ],
        'version': '1.0',
    },
    'CMIP5_Agroclimatic/download_historical.tar.gz')

2022-03-09 18:42:08,483 INFO Welcome to the CDS
2022-03-09 18:42:08,485 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-agroclimatic-indicators
2022-03-09 18:42:08,788 INFO Request is completed
2022-03-09 18:42:08,791 INFO Downloading https://download-0003.copernicus-climate.eu/cache-compute-0003/cache/data6/dataset-sis-agroclimatic-indicators-be62cec8-1478-4edb-b01c-5268a6a71b68.tar.gz to CMIP5_Agroclimatic/download_historical.tar.gz (4.6G)
 11%|█         | 513M/4.59G [03:46<35:26, 2.07MB/s]  

After unzipping the above into `download`, we have two netCDF files for each variable (one for the period 1951- 1980, and one for 1981- 2010). We will read these in and filter them using xarray, in order to turn it into a tabular form aggregated for western MN. 

In [1]:
historical = xr.open_dataset('CMIP5_Agroclimatic/download_historical/BEDD_C3S-glob-agric_gfdl-esm2m_hist_dek_19510101-19801231_v1.1.nc')
historical

In [78]:
tgt_lat = xr.DataArray(np.arange(43.75, 46.75, .5), dims=["lat"])
tgt_lon = xr.DataArray(np.arange(-96.25, -93.75, .5), dims=["lon"])

westMN = historical.sel({
    'lat' : tgt_lat,
    'lon' : tgt_lon,
    'threshold' : 2.0 #to drop when we go back to 1.0
}).drop('time_bounds')

westMN

<xarray.Dataset>
Dimensions:    (time: 1080, lat: 6, lon: 5)
Coordinates:
  * time       (time) datetime64[ns] 1951-01-05 1951-01-15 ... 1980-12-25
  * lat        (lat) float64 43.75 44.25 44.75 45.25 45.75 46.25
  * lon        (lon) float64 -96.25 -95.75 -95.25 -94.75 -94.25
    threshold  float64 2.0
Data variables:
    BEDD       (time, lat, lon) float32 ...
Attributes: (12/14)
    title:          BEDD index with user defined threshold
    institution:    Copernicus Climate Change Service
    source:         Climate projection datasets from the ISIMIP Fast Track pr...
    references:     Tank, A.K., 2010. European Climate Assessment and Dataset...
    comment:        
    history:        Tue Sep 19 10:18:15 2017: cdo -f nc4c -z zip mergetime ta...
    ...             ...
    contact:        https://climate.copernicus.eu
    project:        C3S_422 Global Agriculture
    creation_date:  2019-12-19T08:50:59GMT
    lineage:        
    keywords:       
    license:

Now, we need to cut datetimes into reasonable bins. To do so, we will first summarize into months for each variable, either summing or taking mean for each metric. This is the highest resolution we plan to consider for each variable - even still, we will likely need to aggregate some more to deal with degrees of freedom issues (considering we have only 80-150 target observations for each crop) - but this should be easy as it will be relatively small data by then. 

Since every 3rd observation is the 5th of the month, we can define our months as the 5th to the 5th of each month, and aggregate as such. 

In [94]:
left_bin = westMN.time[np.arange(0,westMN.dims['time'],3)].to_numpy()
westMN.groupby_bins('time', left_bin).groups

{Interval('1951-01-05', '1951-02-05', closed='right'): [0, 1, 2],
 Interval('1951-02-05', '1951-03-05', closed='right'): [3, 4, 5],
 Interval('1951-03-05', '1951-04-05', closed='right'): [6, 7, 8],
 Interval('1951-04-05', '1951-05-05', closed='right'): [9, 10, 11],
 Interval('1951-05-05', '1951-06-05', closed='right'): [12, 13, 14],
 Interval('1951-06-05', '1951-07-05', closed='right'): [15, 16, 17],
 Interval('1951-07-05', '1951-08-05', closed='right'): [18, 19, 20],
 Interval('1951-08-05', '1951-09-05', closed='right'): [21, 22, 23],
 Interval('1951-09-05', '1951-10-05', closed='right'): [24, 25, 26],
 Interval('1951-10-05', '1951-11-05', closed='right'): [27, 28, 29],
 Interval('1951-11-05', '1951-12-05', closed='right'): [30, 31, 32],
 Interval('1951-12-05', '1952-01-05', closed='right'): [33, 34, 35],
 Interval('1952-01-05', '1952-02-05', closed='right'): [36, 37, 38],
 Interval('1952-02-05', '1952-03-05', closed='right'): [39, 40, 41],
 Interval('1952-03-05', '1952-04-05', closed

In [85]:
# westMN.groupby_bins('time', left_bin).sum(dim =['lat','lon'])
aggWestMN = westMN.groupby_bins('time', left_bin).sum(dim =['time', 'lat', 'lon'])
aggWestMN

<xarray.Dataset>
Dimensions:    (time_bins: 359)
Coordinates:
  * time_bins  (time_bins) object (1951-01-05, 1951-02-05] ... (1980-11-05, 1...
    threshold  float64 2.0
Data variables:
    BEDD       (time_bins) float32 0.0 0.0 254.5 ... 7.76e+03 1.569e+03 834.3

In [123]:
any([var in ['BEDD', 'TST'] for var in historical.data_vars])

True

'BEDD'

In [88]:
aggWestMN.to_dataframe().to_csv('historical_out.csv')


This approach seems pretty reasonable, and gives us 360 rows (12 months * 30 years) for each of the ~40 variables we have. The main advantage here has been aggrgating across the entire geographic area in the first place - so now that we can a workflow to do so, we can run it on each variable of interest and concatenate into csv. To this end, the above ingestion and aggregation has been incorporated into `ingest_cmip5.py` and can be run on each variable in the Agroclimatic model.